In [1]:
from dl1_data_handler.reader import DL1DataReaderSTAGE1, DL1DataReaderDL1DH
from ctlearn.utils import setup_DL1DataReader
from ctlearn.data_loader import KerasBatchGenerator
from preprocessing import ImagetoPointCloud

import h5py
import yaml

2024-07-10 12:58:47.157471: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-10 12:58:47.191292: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-10 12:58:47.191322: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-10 12:58:47.191336: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-10 12:58:47.196894: I tensorflow/core/platform/cpu_feature_g

In [7]:
with open('sample_config.yml', 'r') as myconfig:
  config = yaml.load(myconfig, Loader=yaml.Loader)
config["Data"], data_format = setup_DL1DataReader(config, "train")
reader = DL1DataReaderSTAGE1(**config["Data"])
data_len = len(reader)
indices = list(range(data_len))

all_data = KerasBatchGenerator(
    reader,
    indices,
    batch_size=1,
    mode="train",
    class_names=None,
    stack_telescope_images=False,
)

In [17]:
image, labels = all_data.__getitem__(4)
# print(image)
print(labels)
print(image)


[[1. 0.]]
{'images': array([[[[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        ...,

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]]]])}


In [18]:
max_points = 500
relative_coords = True
img_to_pct = ImagetoPointCloud(max_points, relative_coords)
name = "dataset_500_rel_coords.h5"
with h5py.File(f'../data/{name}', 'w') as f:
  for i in range(data_len):
    image, labels = all_data.__getitem__(i)
    # print(image)
    image = image["images"][0,:,:,:]
    labels = labels[0]
    point_cloud = img_to_pct(image)
    grp = f.create_group(f'instance_{i}')
    grp.create_dataset('features', data=point_cloud["features"])
    grp.create_dataset('points', data=point_cloud["points"])
    grp.create_dataset('mask', data=point_cloud["mask"])
    grp.attrs['label'] = labels

print(f"Dataset guardado exitosamente en {name}")

Dataset guardado exitosamente en dataset_500_rel_coords.h5
